In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Data Preprocessing

In [2]:
import pandas as pd

In [123]:
# Read the Redfin data into a DataFrame
file_path_redfin1 = "../data/redfin_data1.csv"
file_path_redfin2 = "../data/redfin_data2.csv"
df_redfin1 = pd.read_csv(file_path_redfin1)
df_redfin2 = pd.read_csv(file_path_redfin2)
df_redfin = pd.concat([df_redfin1, df_redfin2])
df_redfin["period_begin"] = pd.to_datetime(df_redfin["period_begin"])

#make it easier to merge with av_data
df_redfin["date"] = df_redfin["period_begin"] + pd.Timedelta(days = -3)
df_redfin = df_redfin.drop(columns=["period_begin", "period_end"])
df_redfin

,Unnamed: 0,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,...,months_of_supply,months_of_supply_yoy,median_pending_sqft,median_pending_sqft_yoy,average_sale_to_list_ratio,average_sale_to_list_ratio_yoy,median_sale_ppsf,median_sale_ppsf_yoy,State,date
0,2,Crook County,5.0,-0.375000,8.0,0.333333,1.0,-0.500000,9.0,3.500000,...,12.000000,-60.000000,1618.0,-0.071449,0.937917,0.021480,150.000000,0.218853,OR,2017-01-27
1,10,Chambers County,15.0,0.666667,14.0,0.076923,3.0,-0.250000,11.0,0.833333,...,13.363636,-10.469697,2227.0,0.183316,0.973733,-0.015372,94.778849,-0.035945,TX,2017-01-27
2,32,St. Joseph County,12.0,-0.555556,17.0,0.000000,1.0,-0.500000,13.0,-0.133333,...,20.153846,-3.512821,1500.0,-0.417363,0.933705,0.005458,61.458333,0.156474,MI,2017-02-03
3,42,Crook County,6.0,-0.400000,12.0,-0.142857,1.0,-0.500000,1.0,-0.888889,...,107.000000,90.666667,1984.0,0.634267,1.000000,-0.009948,143.509615,0.506180,OR,2017-02-03
4,69,Coos County,16.0,-0.360000,23.0,-0.080000,5.0,-0.166667,11.0,-0.450000,...,28.818182,13.268182,1760.0,0.085415,0.941134,-0.017685,116.892373,-0.041654,OR,2017-03-03
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
166274,4868262,Troup County,15.0,0.363636,20.0,1.857143,1.0,0.000000,18.0,0.200000,...,14.222222,-3.111111,1648.0,-0.208833,0.988753,-0.010559,146.915071,0.086005,GA,2024-11-15
166275,4868265,Nez Perce County,7.0,0.400000,6.0,0.200000,2.0,-0.333333,8.0,1.000000,...,12.375000,-10.875000,2860.0,0.914324,0.999153,0.041135,189.274448,-0.264456,ID,2024-11-15
166276,4868271,Coles County,17.0,1.833333,11.0,0.222222,4.0,3.000000,9.0,2.000000,...,10.555556,-19.777778,1519.5,-0.038291,0.941275,-0.056837,104.166667,0.351781,IL,2024-11-15
166277,4868277,Marion County,6.0,-0.142857,4.0,-0.500000,4.0,1.000000,12.0,1.400000,...,8.166667,-6.433333,1098.5,-0.147127,0.966881,-0.046709,214.650323,-0.145966,IA,2024-11-15


In [124]:
#Inspect Data
df_redfin.info()

<class 'pandas.core.frame.DataFrame'>
Index: 332558 entries, 0 to 166278
Data columns (total 36 columns):
 #   Column                                        Non-Null Count   Dtype         
---  ------                                        --------------   -----         
 0   Unnamed: 0                                    332558 non-null  int64         
 1   region_name                                   332558 non-null  object        
 2   adjusted_average_new_listings                 332558 non-null  float64       
 3   adjusted_average_new_listings_yoy             332558 non-null  float64       
 4   average_pending_sales_listing_updates         332558 non-null  float64       
 5   average_pending_sales_listing_updates_yoy     332558 non-null  float64       
 6   off_market_in_two_weeks                       332558 non-null  float64       
 7   off_market_in_two_weeks_yoy                   332558 non-null  float64       
 8   adjusted_average_homes_sold                   332558 non-nu

In [109]:
# Read the Zillow data into a DataFrame
file_path_zillow_zori = "../data/County_zori_uc_sfrcondomfr_sm_month.csv"
file_path_zillow_zhvi = "../data/County_zhvi_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv"
df_zillow_zori = pd.read_csv(file_path_zillow_zori)
df_zillow_zhvi = pd.read_csv(file_path_zillow_zhvi)

In [110]:
#Extract required features
df_zillow_zori.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)
df_zillow_zhvi.drop(columns=["RegionID", "SizeRank", "StateName", "RegionType", "StateCodeFIPS", "MunicipalCodeFIPS"], inplace=True)

#restructure so that the dates become rows
df_zillow_zori = pd.melt(df_zillow_zori, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZORI")
df_zillow_zhvi = pd.melt(df_zillow_zhvi, id_vars=["RegionName", "State", "Metro"],
                        var_name="Date",
                        value_name="ZHVI")

#apply time horizon
df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)
df_zillow_zhvi = df_zillow_zhvi[df_zillow_zhvi["Date"] > "2015-01-31"]

#merge
df_zillow = pd.merge(df_zillow_zhvi, df_zillow_zori[["RegionName", "State", "Metro", "Date", "ZORI"]],
                     on = ["RegionName", "State", "Metro", "Date"],
                     how = "left")
df_zillow.dropna(inplace=True)

#rename column to match df_redfin
df_zillow.rename(columns={"RegionName": "region_name"}, inplace=True)
df_zillow.rename(columns={"Date": "date"}, inplace=True)

#resample to weekly
df_zillow["date"] += pd.to_timedelta(df_zillow.groupby("date").cumcount(), unit = "s")
df_zillow.set_index("date", inplace=True)
df_zillow = df_zillow.groupby("region_name").resample("W-FRI").ffill()
df_zillow = df_zillow.drop(columns=["region_name"]).reset_index()
df_zillow

<ipython-input-110-a391ab3e0e27>:14: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zori["Date"] = pd.to_datetime(df_zillow_zori["Date"], infer_datetime_format = True)
<ipython-input-110-a391ab3e0e27>:15: UserWarning: The argument 'infer_datetime_format' is deprecated and will be removed in a future version. A strict version of it is now the default, see https://pandas.pydata.org/pdeps/0004-consistent-to-datetime-parsing.html. You can safely remove this argument.
  df_zillow_zhvi["Date"] = pd.to_datetime(df_zillow_zhvi["Date"], infer_datetime_format = True)


,region_name,date,State,Metro,ZHVI,ZORI
0,Ada County,2015-03-06,ID,"Boise City, ID",204657.323819,948.248180
1,Ada County,2015-03-13,ID,"Boise City, ID",204657.323819,948.248180
2,Ada County,2015-03-20,ID,"Boise City, ID",204657.323819,948.248180
3,Ada County,2015-03-27,ID,"Boise City, ID",204657.323819,948.248180
4,Ada County,2015-04-03,ID,"Boise City, ID",205924.714086,956.009824
...,...,...,...,...,...,...
250238,Yuma County,2024-10-04,AZ,"Yuma, AZ",277673.650150,1473.600844
250239,Yuma County,2024-10-11,AZ,"Yuma, AZ",277673.650150,1473.600844
250240,Yuma County,2024-10-18,AZ,"Yuma, AZ",277673.650150,1473.600844
250241,Yuma County,2024-10-25,AZ,"Yuma, AZ",277673.650150,1473.600844


In [112]:
#Inspect Data
df_zillow.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 250243 entries, 0 to 250242
Data columns (total 6 columns):
 #   Column       Non-Null Count   Dtype         
---  ------       --------------   -----         
 0   region_name  250243 non-null  object        
 1   date         250243 non-null  datetime64[ns]
 2   State        250198 non-null  object        
 3   Metro        250198 non-null  object        
 4   ZHVI         250198 non-null  float64       
 5   ZORI         250198 non-null  float64       
dtypes: datetime64[ns](1), float64(2), object(3)
memory usage: 11.5+ MB


In [113]:
#Read alpha vantage data
file_path_av = "../data/av_data.csv"
df_av = pd.read_csv(file_path_av)
df_av["date"] = pd.to_datetime(df_av["date"])
df_av

,date,spy_price,schh_price,unemployment,fed_funds,cpi
0,2016-12-23,198.2226,16.2399,0.047,0.000127,2.41432
1,2016-12-30,196.3081,16.5792,0.047,0.000127,2.41432
2,2017-01-06,199.5399,16.9105,0.047,0.000115,2.42839
3,2017-01-13,199.3994,16.4863,0.047,0.000127,2.42839
4,2017-01-20,199.1272,16.6116,0.047,0.000127,2.42839
...,...,...,...,...,...,...
395,2024-10-11,579.5800,22.5200,0.041,0.000908,3.15664
396,2024-10-18,584.5900,23.1800,0.041,0.000908,3.15664
397,2024-10-25,579.0400,22.7600,0.041,0.000908,3.15664
398,2024-11-01,571.0400,22.1000,0.041,0.000908,3.15664


In [114]:
#Inspect Data
df_av.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          400 non-null    datetime64[ns]
 1   spy_price     400 non-null    float64       
 2   schh_price    400 non-null    float64       
 3   unemployment  400 non-null    float64       
 4   fed_funds     400 non-null    float64       
 5   cpi           400 non-null    float64       
dtypes: datetime64[ns](1), float64(5)
memory usage: 18.9 KB


In [120]:
#merge three data source
df = pd.merge(df_redfin, df_zillow[["region_name", "State", "date" , "ZORI", "ZHVI"]],
                     on = ["region_name", "State", "date"],
                     how = "left")

df = pd.merge(df, df_av[["date", "spy_price", "schh_price", "unemployment", "fed_funds", "cpi"]],
                     on = [ "date"],
                     how = "left")
df.dropna(inplace=True)
df

,region_name,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,...,median_sale_ppsf_yoy,State,date,ZORI,ZHVI,spy_price,schh_price,unemployment,fed_funds,cpi
12,Arapahoe County,162.0,-0.129032,160.0,-0.111111,59.0,0.311111,138.0,-0.126582,318950.0,...,0.097923,CO,2016-12-30,1306.117777,325881.246176,196.3081,16.5792,0.047,0.000127,2.41432
16,Stanislaus County,163.0,0.108844,152.0,-0.061728,52.0,-0.187500,112.0,-0.157895,289000.0,...,0.110304,CA,2017-03-10,1169.349237,270818.216906,208.7437,16.0218,0.044,0.000127,2.43801
18,Stearns County,73.0,0.123077,42.0,0.000000,11.0,0.100000,31.0,0.033333,168500.0,...,0.111337,MN,2017-03-10,890.363833,186465.879784,208.7437,16.0218,0.044,0.000127,2.43801
25,Washoe County,73.0,-0.401639,119.0,-0.201342,11.0,-0.312500,85.0,-0.174757,313500.0,...,0.051403,NV,2016-12-30,1148.605311,305904.466295,196.3081,16.5792,0.047,0.000127,2.41432
26,Chittenden County,32.0,0.142857,26.0,-0.277778,5.0,0.000000,23.0,-0.148148,302450.0,...,-0.103745,VT,2017-01-06,1714.842956,304940.863948,199.5399,16.9105,0.047,0.000115,2.42839
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
332534,Cuyahoga County,397.0,-0.076744,405.0,-0.093960,150.0,-0.202128,292.0,-0.165714,202950.0,...,0.108241,OH,2024-06-14,1339.131151,199743.798643,539.3754,19.6503,0.041,0.000999,3.14175
332536,Butler County,14.0,-0.300000,15.0,-0.062500,6.0,0.200000,20.0,0.250000,243400.0,...,-0.013492,KS,2024-08-09,1139.957933,239090.344791,531.3578,21.5503,0.042,0.000999,3.14796
332539,Schuylkill County,40.0,0.333333,34.0,0.096774,12.0,-0.076923,30.0,0.200000,124950.0,...,-0.009550,PA,2024-06-14,1017.609822,138286.398240,539.3754,19.6503,0.041,0.000999,3.14175
332540,Aransas County,14.0,0.400000,14.0,0.555556,0.0,-1.000000,16.0,1.000000,389500.0,...,0.310313,TX,2024-06-14,1335.259605,328197.370747,539.3754,19.6503,0.041,0.000999,3.14175


In [121]:
df.describe()

,adjusted_average_new_listings,adjusted_average_new_listings_yoy,average_pending_sales_listing_updates,average_pending_sales_listing_updates_yoy,off_market_in_two_weeks,off_market_in_two_weeks_yoy,adjusted_average_homes_sold,adjusted_average_homes_sold_yoy,median_new_listing_price,median_new_listing_price_yoy,...,median_sale_ppsf,median_sale_ppsf_yoy,date,ZORI,ZHVI,spy_price,schh_price,unemployment,fed_funds,cpi
count,130956.000000,130956.000000,130956.000000,130956.000000,130956.000000,130956.000000,130956.000000,130956.000000,1.309560e+05,130956.000000,...,130956.000000,130956.000000,130956,130956.000000,1.309560e+05,130956.000000,130956.000000,130956.000000,130956.000000,130956.000000
mean,143.092985,0.022301,145.242898,0.009385,38.962094,0.233339,122.010385,0.010499,3.738457e+05,0.080076,...,211.170334,0.079318,2021-08-12 09:05:02.501603328,1574.053351,3.524350e+05,373.920315,18.968880,0.044152,0.000501,2.810539
min,1.000000,-0.963964,1.000000,-0.995088,0.000000,-1.000000,1.000000,-0.961538,2.657100e+04,-0.930806,...,16.846484,-0.906676,2016-12-30 00:00:00,566.539368,7.844892e+04,196.308100,12.331300,0.034000,0.000008,2.414320
25%,37.000000,-0.172414,38.000000,-0.181818,7.000000,-0.333333,33.000000,-0.197691,2.400000e+05,-0.003958,...,133.643214,0.012971,2019-09-06 00:00:00,1203.825701,2.228080e+05,269.298200,17.207800,0.036000,0.000142,2.563940
50%,77.000000,-0.017857,80.000000,-0.034507,17.000000,0.000000,68.000000,-0.035714,3.224500e+05,0.066700,...,171.428571,0.069056,2022-01-14 00:00:00,1468.353450,3.000463e+05,388.812900,18.951300,0.038000,0.000417,2.811480
75%,164.000000,0.148148,168.000000,0.126984,44.000000,0.400000,142.000000,0.143541,4.374950e+05,0.149911,...,230.263158,0.135843,2023-09-01 00:00:00,1825.074708,4.153175e+05,439.084500,20.279800,0.043000,0.000953,3.067460
max,2608.000000,22.818182,2649.000000,154.000000,1165.000000,57.000000,2679.000000,16.300000,1.075000e+07,8.673529,...,3308.823529,5.984058,2024-11-01 00:00:00,16914.169881,2.442129e+06,584.590000,24.310300,0.148000,0.000999,3.156640
std,214.627198,0.366387,215.613987,0.661803,64.424780,1.353367,178.412213,0.366924,2.242917e+05,0.165043,...,146.302825,0.123573,NaN,622.943580,2.054568e+05,105.482901,2.114577,0.017041,0.000389,0.255577
